## Example notebook to demonstrate scaling up clusters

This notebook only illustrates how to list/get/scale `sparkclusters`. The process is the same for `rayclusters` and `daskclusters`.

Make the approrpriate change to the `kind` in the url and run the same notebook to try this out for ray and dask clusters

In [ ]:
import requests
import os

In [ ]:
base_url_of_cluster_scaler = "http://ddl-cluster-scaler-svc.domino-field.svc.cluster.local"
service_name="ddl_cluster_scaler"

In [ ]:
# Is service running
healthz_url = f"{base_url_of_cluster_scaler}/healthz"
requests.get(healthz_url).text

In [ ]:
def get_auth_headers():
    domino_api_proxy= os.environ['DOMINO_API_PROXY']
    #This is a proxy for the service account tokem
    token = requests.get(f"{domino_api_proxy}/access-token").text
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"   # add this if you're posting JSON
    }
    return headers 

In [ ]:
# List all visible clusters
cluster_kind="sparkclusters"
list_cluster_url = f"{base_url_of_cluster_scaler}/{service_name}/list/{cluster_kind}"
print(list_cluster_url)
resp = requests.get(list_cluster_url,headers=get_auth_headers())
print(f"Status code {resp.status_code}")
all_clusters = resp.json()
#print(all_clusters)

In [ ]:
# Get the spark id
spark_cluster_id = f"spark-{os.environ['DOMINO_RUN_ID']}"

In [ ]:
# Get the spark details
cluster_kind="sparkclusters"
get_cluster_url = f"{base_url_of_cluster_scaler}/{service_name}/get/{cluster_kind}/{spark_cluster_id}"
print(get_cluster_url)
resp = requests.get(get_cluster_url,headers=get_auth_headers())
print(f"Status code {resp.status_code}")
cluster_details = resp.json()

In [ ]:
# Scale up the cluster
cluster_kind="sparkclusters"
scale_cluster_url = f"{base_url_of_cluster_scaler}/{service_name}/scale/{cluster_kind}/{spark_cluster_id}"
print(scale_cluster_url)
replicas=3
resp = requests.post(
    scale_cluster_url,
    headers=get_auth_headers(),
    json={"replicas": replicas},
    timeout=(3.05, 10),
)
print(f"Status code {resp.status_code}")
cluster_details = resp.json()

In [ ]:
# Scale down the cluster
cluster_kind="sparkclusters"
scale_cluster_url = f"{base_url_of_cluster_scaler}/{service_name}/scale/{cluster_kind}/{spark_cluster_id}"
print(scale_cluster_url)
replicas=1
resp = requests.post(
    scale_cluster_url,
    headers=get_auth_headers(),
    json={"replicas": replicas},
    timeout=(3.05, 10),
)
print(f"Status code {resp.status_code}")
cluster_details = resp.json()